In [5]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers    import Activation , Dense , Dropout
from tensorflow.keras.optimizers import SGD


In [6]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("D:\.folder e\work space\projects\project 5\intents.json").read())

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Mohand\AppData\Local\Temp\ipykernel_11616\2770469948.py:2: SyntaxWarning: invalid escape sequence '\.'
  intents = json.loads(open("D:\.folder e\work space\projects\project 5\intents.json").read())


In [7]:
words = []
documents = []
classes = []
ignore_letters = ["!","?",".",","]

In [4]:
for intent in intents["intents"]:
    for pattern in intent['patterns']:
        word_tokenize = nltk.word_tokenize(pattern)
        words.extend(word_tokenize)
        documents.append((word_tokenize  , intent["tag"]))
        if intent["tag"] not in classes :
            classes .append(intent["tag"])

In [5]:
words = [lemmatizer.lemmatize(word) for word in words if word not in  ignore_letters]
words=sorted(set(words))
classes = sorted(set(classes))


In [6]:
pickle.dump(words ,open("words.pkl" , "wb") )
pickle.dump(classes ,open("classes.pkl" , "wb") )

In [7]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)


In [ ]:

training = np.array(training, dtype=object)
train_x = np.array([np.array(x) for x in training[:, 0]], dtype=np.float32)
train_y = np.array([np.array(y) for y in training[:, 1]], dtype=np.float32)


model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)



Epoch 1/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0298 - loss: 2.6629    
Epoch 2/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0361 - loss: 2.6614     
Epoch 3/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2434 - loss: 2.5566
Epoch 4/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - accuracy: 0.1958 - loss: 2.6295
Epoch 5/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1476 - loss: 2.5054 
Epoch 6/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2311 - loss: 2.4753
Epoch 7/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - accuracy: 0.2719 - loss: 2.4793
Epoch 8/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - accuracy: 0.1814 - loss: 2.4617
Epoch 9/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1921 - loss: 2.4483.50
Epoch 10/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - accuracy: 0.2264 - loss: 2.3477
Epoch 11/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - accuracy: 0.2622 - loss: 2.2629 
Epoch 12/300
13/13 ━━━━━━━━━━━━━━━━━━

In [15]:
model.save('chatbot_model.keras')